In [ ]:
from sqlalchemy import create_engine

server = "localhost"
database = "Bokhandel"
driver = "ODBC Driver 17 for SQL Server"
engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

with engine.connect() as connection:
    print(f"Successfully connected to {database} on server {server}")


Successfully connected to Bokhandel on server localhost


In [9]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, CHAR
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

# Anslut med användaren Bokhandel,
server = "localhost"
database = "Bokhandel"
driver = "ODBC Driver 17 for SQL Server"
engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class Böcker(Base):
    __tablename__ = 'Böcker'
    ISBN13 = Column(CHAR(13), primary_key=True)
    Titel = Column(String)


class Butiker(Base):
    __tablename__ = 'Butiker'
    ButikID = Column(Integer, primary_key=True)
    Butiksnamn = Column(String)
    Ort = Column(String)

class LagerSaldo(Base):
    __tablename__ = 'LagerSaldo'
    ButikID = Column(Integer, ForeignKey('Butiker.ButikID'), primary_key=True)
    ISBN = Column(CHAR(13), ForeignKey('Böcker.ISBN13'), primary_key=True)
    Antal = Column(Integer)
    butik = relationship('Butiker')
    bok = relationship('Böcker')

# Fritextsökning på boktitel,
search_term = input("Ange sökord för boktitel: ")

results = (
    session.query(Böcker.Titel, Butiker.Butiksnamn, Butiker.Ort, LagerSaldo.Antal) 
    .join(LagerSaldo, LagerSaldo.ISBN == Böcker.ISBN13)
    .join(Butiker, Butiker.ButikID == LagerSaldo.ButikID)
    .filter(Böcker.Titel.ilike(f"%{search_term}%"))
    .all()
)

if results:
    for titel, butik, ort, antal in results:
        print(f"Bok: {titel}, Butik: {butik} i {ort}, Antal: {antal}")
else:
    print("Inga böcker hittades.")

session.close()

Bok: Ronja Rövardotter, Butik: Jennys Hörna i Stockholm, Antal: 8
Bok: Ronja Rövardotter, Butik: Jennys Hörna i Göteborg, Antal: 5
Bok: Ronja Rövardotter, Butik: Jennys Hörna i Malmö, Antal: 9
